In [43]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
solve_season = '2024-25'
solve_gameweek = 22
load_projections_from_file = True

In [45]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Loaded projections from CSV file.


### Player Manipulation

### Player Force/Banning

In [46]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Matches found:
1. Fabianski (ID: 521, Index: 310, Score: 100)
2. Kabia (ID: 659, Index: 373, Score: 72)
3. Aina (ID: 422, Index: 257, Score: 68)
4. Skipp (ID: 501, Index: 298, Score: 68)
5. Ings (ID: 523, Index: 311, Score: 68)
6. Disasi (ID: 167, Index: 96, Score: 60)
7. Barnes (ID: 392, Index: 239, Score: 60)
8. Kinsky (ID: 716, Index: 408, Score: 60)
9. Zabarnyi (ID: 85, Index: 54, Score: 59)
10. Mings (ID: 49, Index: 32, Score: 54)
Banned: Fabianski (ID: 521, Index: 310)
Matches found:
1. Ward (ID: 308, Index: 183, Score: 100)
2. Ødegaard (ID: 13, Index: 8, Score: 68)
3. Trossard (ID: 23, Index: 17, Score: 68)
4. Damsgaard (ID: 89, Index: 57, Score: 68)
5. Nørgaard (ID: 101, Index: 66, Score: 68)
6. Edouard (ID: 198, Index: 112, Score: 68)
7. Vestergaard (ID: 307, Index: 182, Score: 68)
8. Bernardo (ID: 342, Index: 206, Score: 68)
9. Gvardiol (ID: 350, Index: 212, Score: 68)
10. Stewart (ID: 473, Index: 283, Score: 68)
Banned: Ward (ID: 308, Index: 183)
Matches found:
1. Raúl (ID: 

In [47]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW22 Challenge: Restricted Budget - Budget is restricted to 25 million - spend wisely

### Optimisation

In [48]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# 1-2 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 2

# 1-2 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 2

# 1-2 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) <= 2

# Budget constraint of 25m
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 25

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")
        
solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Sánchez - Chelsea - Cost: 4.8m - Predicted Points: 4.23

Defender:
  Murillo - Nott'm Forest - Cost: 4.6m - Predicted Points: 4.49
  N.Williams - Nott'm Forest - Cost: 4.3m - Predicted Points: 4.49

Midfielder:
  Amad (C) - Man Utd - Cost: 5.5m - Predicted Points: 11.16

Forward:
  João Pedro - Brighton - Cost: 5.7m - Predicted Points: 4.91

Total Predicted Points: 29.28
Total Cost: 24.9m


In [49]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

projections_data.to_csv('gameweek_projections.csv', index=False)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
106,185,Sánchez,Chelsea,200,Goalkeeper,4.8,4.23,90,Wolves (H),0.0,0.0,0.0,0.357,0.0
269,443,Sels,Nott'm Forest,21,Goalkeeper,5.0,4.19,90,Southampton (H),0.0,0.0,0.0,0.524,0.0
10,15,Raya,Arsenal,200,Goalkeeper,5.6,4.16,90,Aston Villa (H),0.0,0.0,0.0,0.476,0.0
185,310,A.Becker,Liverpool,30,Goalkeeper,5.4,3.92,90,Brentford (A),0.0,0.0,0.0,0.364,0.0
136,235,Pickford,Everton,241,Goalkeeper,5.0,3.83,90,Spurs (H),0.0,0.0,0.0,0.231,0.0
183,308,Ward,Leicester,244,Goalkeeper,3.9,3.71,90,Fulham (H),0.0,0.0,0.0,0.041,0.0
408,716,Kinsky,Spurs,57,Goalkeeper,4.5,3.69,90,Everton (A),0.0,0.0,0.0,0.267,0.0
310,521,Fabianski,West Ham,172,Goalkeeper,4.1,3.66,90,Crystal Palace (H),0.0,0.0,0.0,0.250,0.0
147,248,Leno,Fulham,80,Goalkeeper,5.0,3.61,90,Leicester (A),0.0,0.0,0.0,0.364,0.0
210,347,Ederson M.,Man City,30,Goalkeeper,5.3,3.60,79,Ipswich (A),0.0,0.0,0.0,0.000,0.0


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
1,3,Gabriel,Arsenal,30,Defender,6.4,5.02,90,Aston Villa (H),0.060,0.080,0.135,0.476,0.086
335,573,Milenković,Nott'm Forest,190,Defender,4.8,4.70,90,Southampton (H),0.125,0.030,0.152,0.524,0.141
186,311,Alexander-Arnold,Liverpool,241,Defender,7.2,4.64,89,Brentford (A),0.124,0.311,0.396,0.364,0.139
266,436,Murillo,Nott'm Forest,30,Defender,4.6,4.49,90,Southampton (H),0.006,0.007,0.013,0.491,0.008
267,437,N.Williams,Nott'm Forest,244,Defender,4.3,4.49,88,Southampton (H),0.103,0.062,0.159,0.524,0.115
212,350,Gvardiol,Man City,97,Defender,6.0,4.41,90,Ipswich (A),0.133,0.146,0.260,0.421,0.150
257,422,Aina,Nott'm Forest,157,Defender,5.3,4.41,90,Southampton (H),0.091,0.036,0.124,0.524,0.103
13,18,Saliba,Arsenal,73,Defender,6.3,4.17,90,Aston Villa (H),0.067,0.057,0.119,0.476,0.077
3,6,J.Timber,Arsenal,152,Defender,5.7,4.12,90,Aston Villa (H),0.056,0.103,0.153,0.446,0.064
200,335,Robertson,Liverpool,243,Defender,5.9,4.08,79,Brentford (A),0.081,0.197,0.262,0.364,0.092


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
104,182,Palmer,Chelsea,241,Midfielder,11.4,8.24,90,Wolves (H),0.524,0.372,0.701,0.381,0.783
196,328,M.Salah,Liverpool,63,Midfielder,13.7,7.92,90,Brentford (A),0.508,0.534,0.771,0.364,0.740
222,364,Amad,Man Utd,54,Midfielder,5.5,5.58,89,Brighton (H),0.216,0.100,0.295,0.294,0.258
208,345,De Bruyne,Man City,21,Midfielder,9.4,5.50,72,Ipswich (A),0.035,0.030,0.063,0.395,0.044
211,348,Foden,Man City,241,Midfielder,9.2,5.38,83,Ipswich (A),0.374,0.311,0.569,0.421,0.506
223,366,B.Fernandes,Man Utd,173,Midfielder,8.4,5.28,87,Brighton (H),0.222,0.204,0.381,0.276,0.270
244,398,Gordon,Newcastle,241,Midfielder,7.7,5.16,82,Bournemouth (H),0.346,0.168,0.456,0.364,0.463
339,585,I.Sarr,Crystal Palace,189,Midfielder,5.9,4.99,90,West Ham (A),0.250,0.109,0.332,0.267,0.303
64,99,Mbeumo,Brentford,38,Midfielder,7.8,4.90,90,Liverpool (H),0.278,0.093,0.345,0.100,0.340
113,199,Eze,Crystal Palace,241,Midfielder,6.6,4.87,82,West Ham (A),0.208,0.067,0.261,0.250,0.253


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
213,351,Haaland,Man City,161,Forward,14.8,7.35,90,Ipswich (A),0.591,0.218,0.680,0.421,0.943
246,401,Isak,Newcastle,206,Forward,9.4,7.09,83,Bournemouth (H),0.511,0.112,0.566,0.364,0.777
272,447,Wood,Nott'm Forest,154,Forward,6.9,6.20,89,Southampton (H),0.431,0.058,0.464,0.524,0.603
102,180,N.Jackson,Chelsea,189,Forward,8.0,5.19,80,Wolves (H),0.379,0.102,0.442,0.357,0.530
2,4,Havertz,Arsenal,80,Forward,7.8,5.13,87,Aston Villa (H),0.278,0.141,0.380,0.446,0.354
321,541,Cunha,Wolves,30,Forward,7.0,5.06,90,Chelsea (A),0.231,0.094,0.304,0.085,0.284
150,252,Raúl,Fulham,139,Forward,5.7,5.00,78,Leicester (A),0.333,0.048,0.365,0.341,0.441
76,129,João Pedro,Brighton,30,Forward,5.7,4.91,90,Man Utd (A),0.068,0.039,0.105,0.200,0.083
117,207,Mateta,Crystal Palace,73,Forward,7.3,4.88,84,West Ham (A),0.291,0.063,0.335,0.267,0.365
70,110,Wissa,Brentford,50,Forward,6.2,4.81,87,Liverpool (H),0.223,0.039,0.253,0.100,0.269
